In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/jageorge/Downloads/Run_2_PostedWorkOrders_11-Jan-21 23_31_06.csv')
df.head()

,Work Order Name,Post Submission Time,Post Start Time,Post Complete Time,Networks Or Edits,Bin,Operation
0,J0_02026,12/01/2021 06:04:59,11/01/2021 23:30:55,11/01/2021 23:31:07,2,1,EDIT
1,J2_00805,09/01/2021 00:58:40,11/01/2021 23:31:07,11/01/2021 23:31:18,1,1,CREATE
2,J0_08309,09/01/2021 00:58:50,11/01/2021 23:31:19,11/01/2021 23:31:27,3,1,EDIT
3,J0_08317,09/01/2021 00:58:51,11/01/2021 23:31:27,11/01/2021 23:31:35,1,1,EDIT
4,J0_07124,09/01/2021 00:58:54,11/01/2021 23:31:36,11/01/2021 23:31:45,1,1,EDIT


In [3]:
df.dtypes
df['PostSubTime'] = pd.to_datetime(df['Post Submission Time'] )
df['PostStartTime'] = pd.to_datetime(df['Post Start Time'] )
df['PostCompTime'] = pd.to_datetime(df['Post Complete Time'] )

In [4]:
df = df[['Bin','PostSubTime','PostStartTime','PostCompTime','Operation']]
df.head()

,Bin,PostSubTime,PostStartTime,PostCompTime,Operation
0,1,2021-12-01 06:04:59,2021-11-01 23:30:55,2021-11-01 23:31:07,EDIT
1,1,2021-09-01 00:58:40,2021-11-01 23:31:07,2021-11-01 23:31:18,CREATE
2,1,2021-09-01 00:58:50,2021-11-01 23:31:19,2021-11-01 23:31:27,EDIT
3,1,2021-09-01 00:58:51,2021-11-01 23:31:27,2021-11-01 23:31:35,EDIT
4,1,2021-09-01 00:58:54,2021-11-01 23:31:36,2021-11-01 23:31:45,EDIT


In [5]:
df['WeightTime'] = df['PostStartTime'].shift(periods=-1) - df['PostCompTime'] # need to shift to subtract last complete time
df['AvPostTime'] = df['PostCompTime'] - df['PostStartTime']
df['PostStartHour'] = df.PostStartTime.dt.hour
df.head()

,Bin,PostSubTime,PostStartTime,PostCompTime,Operation,WeightTime,AvPostTime,PostStartHour
0,1,2021-12-01 06:04:59,2021-11-01 23:30:55,2021-11-01 23:31:07,EDIT,00:00:00,00:00:12,23
1,1,2021-09-01 00:58:40,2021-11-01 23:31:07,2021-11-01 23:31:18,CREATE,00:00:01,00:00:11,23
2,1,2021-09-01 00:58:50,2021-11-01 23:31:19,2021-11-01 23:31:27,EDIT,00:00:00,00:00:08,23
3,1,2021-09-01 00:58:51,2021-11-01 23:31:27,2021-11-01 23:31:35,EDIT,00:00:01,00:00:08,23
4,1,2021-09-01 00:58:54,2021-11-01 23:31:36,2021-11-01 23:31:45,EDIT,00:00:00,00:00:09,23


In [6]:
df[df.PostStartHour == 0].AvPostTime.mean()

Timedelta('0 days 00:00:12.830324')

In [7]:
first = df.groupby(df.PostStartHour).agg({'Bin':['count'],
                                  'WeightTime':lambda x: x.astype('timedelta64[s]').mean(),
                                 'AvPostTime':lambda x: x.astype('timedelta64[s]').mean()})
first.columns = ['Bin','AvgWeighTime','AvgPostTime']
first = first.reset_index
first

<bound method DataFrame.reset_index of                Bin  AvgWeighTime   AvgPostTime
PostStartHour                                 
0              277      0.180505     12.830325
1              246      0.219512     14.414634
2              226      0.424779     15.504425
3              228      0.421053     15.381579
4              230      0.400000     15.252174
5              175      1.297143     19.297143
6              164      1.987805     20.000000
7              213      0.342723     16.582160
8              199      0.402010     17.623116
9              200      0.245000     17.745000
10             171      0.169591     20.847953
11             181      1.165746     18.856354
12             130      2.484615     25.376923
13             105      0.711538     20.600000
23             150      0.393333  16715.240000>

In [8]:
df.groupby('Operation').agg({'AvPostTime':lambda x: x.astype('timedelta64[s]').mean()})

,AvPostTime
Operation,
CREATE,32.110619
EDIT,1039.736390


In [74]:
createtotal = df[df.Operation=='CREATE']['Bin'].count()
edittotal = df[df.Operation=='EDIT']['Bin'].count()
print(createtotal,edittotal)

lst = df.groupby(['Operation','PostStartHour'])['Bin'].count()
lst = lst.reset_index()
lst


452 2443


,Operation,PostStartHour,Bin
0,CREATE,0,51
1,CREATE,1,47
2,CREATE,2,35
3,CREATE,3,29
4,CREATE,4,33
5,CREATE,5,28
6,CREATE,6,26
7,CREATE,7,19
8,CREATE,8,36
9,CREATE,9,31


In [75]:
def funct(row):
    if row['Operation']=='CREATE':
        return row['Bin']/createtotal
    else:
        return row['Bin']/edittotal

In [76]:
lst['Percent'] = lst.apply(lambda row: funct(row), axis=1)
lst

,Operation,PostStartHour,Bin,Percent
0,CREATE,0,51,0.112832
1,CREATE,1,47,0.103982
2,CREATE,2,35,0.077434
3,CREATE,3,29,0.064159
4,CREATE,4,33,0.073009
5,CREATE,5,28,0.061947
6,CREATE,6,26,0.057522
7,CREATE,7,19,0.042035
8,CREATE,8,36,0.079646
9,CREATE,9,31,0.068584


In [78]:
df.groupby(['PostStartHour','Operation'])['Bin'].count()

PostStartHour  Operation
0              CREATE        51
               EDIT         226
1              CREATE        47
               EDIT         199
2              CREATE        35
               EDIT         191
3              CREATE        29
               EDIT         199
4              CREATE        33
               EDIT         197
5              CREATE        28
               EDIT         147
6              CREATE        26
               EDIT         138
7              CREATE        19
               EDIT         194
8              CREATE        36
               EDIT         163
9              CREATE        31
               EDIT         169
10             CREATE        42
               EDIT         129
11             CREATE        18
               EDIT         163
12             CREATE        19
               EDIT         111
13             CREATE        20
               EDIT          85
23             CREATE        18
               EDIT         132
Name: Bin, dtyp